<a href="https://colab.research.google.com/github/assisav4/deteccao_fraudes/blob/main/Detec%C3%A7%C3%A3o_de_transa%C3%A7%C3%B5es_fraudulentas_%5BML%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
chitwanmanchanda_fraudulent_transactions_data_path = kagglehub.dataset_download('chitwanmanchanda/fraudulent-transactions-data')

print('Data source import complete.')


---
# **Análise de Fraude Finaceira**
---

Fraudes em transações financeiras são uma preocupação crescente para bancos, empresas de cartão de crédito, e consumidores. Estimativas sugerem que bilhões de dólares são perdidos anualmente devido a atividades fraudulentas, como clonagem de cartões, phishing, e outras formas de fraude digital. A aplicação de análise de dados e machine learning para a prevenção de fraudes oferece uma possivel solução para um problema que afeta milhões de pessoas em todo o mundo.

OBS:
este projeto tem como objetivo construir um modelo simples de machine learning para prever fraudes financeiras este nootebook esta suscetível
a erros e metodos ineficientes.

# **Objetivo**


Desenvolver um modelo preditivo baseado em machine learning capaz de detectar fraudes em transações financeiras em tempo real, minimizando falsos positivos e negativos, a fim de proteger consumidores e instituições financeiras de perdas econômicas e aumentar a segurança das transações digitais.

dados obtiveis no [kaggle](https://www.kaggle.com/datasets/chitwanmanchanda/fraudulent-transactions-data/data)

# **1. Importando Bibliotecas**





In [1]:
#analise da dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#machine learnig
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


# **2.  Obtendo e conhecendo os dados**

 **Legenda Dos Dados**

**step** - mapeia uma unidade de tempo no mundo real. Neste caso, 1 passo equivale a 1 hora. Total de passos 744 (simulação de 30 dias).


---


**type** - tipos de pagamento: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER.


---


**amount** - valor da transação em moeda local.


---


**nameOrig** - cliente que iniciou a transação


---


**oldbalanceOrg** - antigo saldo antes da transação


---


**newbalanceOrig** - novo saldo após a transação


---


**nameDest** - cliente que é o destinatário da transação

---

**oldbalanceDest** - destinatário do saldo inicial antes da transação. Observe que não há informações para clientes que iniciam com M (Comerciantes).

---

**newbalanceDest** - novo destinatário do saldo após a transação. Observe que não há informações para clientes que iniciam com M (Comerciantes).

---

**isFraud** - São as transações realizadas pelos agentes fraudulentos dentro da simulação. Neste conjunto de dados específico, o comportamento fraudulento dos agentes visa lucrar assumindo o controle das contas dos clientes e tentando esvaziar os fundos transferindo para outra conta e depois sacando do sistema.


---


**isFlaggedFraud** - O modelo de negócio visa controlar transferências massivas de uma conta para outra e sinalizar tentativas ilegais. Uma tentativa ilegal neste conjunto de dados é uma tentativa de transferir mais de 200.000 em uma única transação.

In [2]:
data=pd.read_csv("/kaggle/input/fraudulent-transactions-data/Fraud.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/fraudulent-transactions-data/Fraud.csv'

> verifico os metodos info() e discribe() para me introduzir aos dados

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include=[object])

> verifico a existência de nulos


In [ ]:
data.isnull().sum()

# **3. Analise exploratória de dados (EDA)**

In [ ]:
#nome do destinatário
dm=data[data['nameDest'].str.startswith('M')]#comerciante

dc=data[data['nameDest'].str.startswith('C')]#cliente/pessoa comun

#contagem e comparação com isFraud

dm_fraud=dm[dm['isFraud']==1].value_counts()
dc_fraud=dc[dc['isFraud']==1].value_counts()

dm_fraud_count=dm_fraud.sum()
dc_fraud_count=dc_fraud.sum()

tamanho_dm = dm.shape[0]
tamanho_dc = dc.shape[0]

print(f"número de fraudes com destino a comercio {dm_fraud_count}, quantia de comerciantes totais {tamanho_dm}")
print(f"número de fraudes com destino a cliente {dc_fraud_count}, quantia de clientes totais {tamanho_dc}")


> percebe-se que não a fraudes envolvendo comerciantes porem isso pode ser apenas uma caracteristica deste dataframe em questão então para manter o modelo de machine learning generico o mesmo não incluira tal variavel.

In [ ]:
#step x amount
with sns.axes_style("darkgrid"):
  plt.figure(figsize=(25, 8))
  sns.lineplot(data=data,x="step",y="amount",color="red")



> o grafico não demonstra grande variância no periodo de tempo recordado pela falta de uma data de inicio de coleta dos dados a conversão para uma data utilizavel poderia induzir o modelo de machine learnig a um erro, mais ainda sim deixarei essa variavel no modelo como uma linha do tempo.

In [ ]:
#transferencias por tipo
x=data.type.value_counts()
with sns.axes_style("darkgrid"):
  colors = sns.color_palette('pastel')[0:5]
  labels = ["CASH_OUT", "PAYMENT", "CASH_IN", "TRANSFER", "DEBIT"]
  plt.figure(figsize=(14, 8))
  plt.pie(x, labels = labels, colors = colors, autopct='%.0f%%',center=(1,2))
  plt.title("Tipo de transação")

  plt.show()

> analisa-se apartir deste grafico de pizza que os tipos de pagamento são relativamente enquilibrado tirando o debit e transfer os demais mantem um certo inquilibrio nas transações.

In [ ]:
# tipos de transações mais utilizados por fraudadores
fraud_type = data[data['isFraud'] == 1]['type'].value_counts()
print(fraud_type)

with sns.axes_style("darkgrid"):
  colors = sns.color_palette('pastel')[0:2]
  labels = ["CASH_OUT", "TRANSFER"]
  plt.figure(figsize=(14, 8))
  plt.pie(fraud_type, labels = labels, colors = colors, autopct='%.2f%%',center=(1,2))
  plt.title("Tipo de transação fraudulenta")

  plt.show()


> é visivel que existe um metodo preferivel para os agentes fraudulentos sendo eles o transfer e o cash_out que seriam respectivamente transferencia e sacando num caixa eletronico .

In [ ]:
#contagems de fraude e porcentagem arredondada

fraud_contagem = data.isFraud.value_counts()
fraud_porcentagem = round(data.isFraud.value_counts()/len(data)*100,2)

fraud_r = pd.DataFrame({
    'contagem': fraud_contagem,
    'porcentual': fraud_porcentagem
})

fraud_r


> percebe-se que o número de fraude é bem pequeno comparando com o numero total de transações.

In [ ]:
data["isFlaggedFraud"].value_counts() #analise da variavel flaggedfraud

In [ ]:
data.query("isFraud==1 and isFlaggedFraud==1").shape[0]

>a variavel isFlaggedFraud possue 16 aparições "positiva" e todas elas coecidem com uma fraude porem por ser tão rara não sera incluido no modelo.

# **4. procesamento das variaveis**

In [ ]:
#removendo colunas que não serão usadas
data_limpo=data.drop(["nameOrig",
                      "isFlaggedFraud",
                      "nameDest"],axis=1)

In [ ]:
#procesamento da coluna type

data_limpo['type_cash_out_nom'] = data_limpo['type'].apply(lambda type: 1 if type == 'CASH_OUT' else 0)
data_limpo["type_cash_in_nom"]=data_limpo['type'].apply(lambda type: 1 if type == 'CASH_IN' else 0)
data_limpo["type_debit_nom"]=data_limpo['type'].apply(lambda type: 1 if type == 'DEBIT' else 0)
data_limpo["type_payment_nom"]=data_limpo['type'].apply(lambda type: 1 if type == 'PAYMENT' else 0)
data_limpo["type_transfer_nom"]=data_limpo['type'].apply(lambda type: 1 if type == 'TRANSFER' else 0)


In [ ]:
#procesamento das colunas numéricas padronização

#amount
mean = data['amount'].mean()
std = data['amount'].std()

data_limpo['amount_std'] = data['amount'].apply(lambda x: (x - mean) / std)

#step
mean = data['step'].mean()
std = data['step'].std()

data_limpo['step_std'] = data['step'].apply(lambda x: (x - mean) / std)

#oldbalanceorg
mean = data['oldbalanceOrg'].mean()
std = data['oldbalanceOrg'].std()

data_limpo['oldbalanceOrg_std'] = data['oldbalanceOrg'].apply(lambda x: (x - mean) / std)

#newbalanceorg
mean = data['newbalanceOrig'].mean()
std = data['newbalanceOrig'].std()

data_limpo['newbalanceOrig_std'] = data['newbalanceOrig'].apply(lambda x: (x - mean) / std)

#oldbalancedest
mean = data['oldbalanceDest'].mean()
std = data['oldbalanceDest'].std()

data_limpo['oldbalanceDest_std'] = data['oldbalanceDest'].apply(lambda x: (x - mean) / std)

#newbalancedest
mean = data['newbalanceDest'].mean()
std = data['newbalanceDest'].std()

data_limpo['newbalanceDest_std'] = data['newbalanceDest'].apply(lambda x: (x - mean) / std)


In [ ]:
#removendo colunas antigas
data_limpo=data_limpo.drop(["type",
                            "step",
                            "amount",
                            "oldbalanceOrg",
                            "newbalanceOrig",
                            "oldbalanceDest",
                            "newbalanceDest"],axis=1)

In [ ]:
# mapa de calor comparando as correlações
plt.figure(figsize=(12, 8))  # Define o tamanho da figura
sns.heatmap(data_limpo.corr(), annot=True, cmap='coolwarm', fmt=".2f") #caucula a correlação e cria o grafico
plt.title('Mapa de Calor da Correlação') #titulo
plt.show()


In [ ]:
data_limpo.head()

> data frame preparado :)


# **5. Criação e treinamento do modelo de arvore de decisão**

In [ ]:
#definição do modelo
modelo = DecisionTreeClassifier()

In [ ]:
#divisaõ do treinamento e teste
predictors_train, predictors_test, target_train, target_test = train_test_split(
    data_limpo.drop(['isFraud'], axis=1),
    data_limpo['isFraud'],
    test_size=1/3,
    random_state=109
)

In [ ]:
#treinamento do modelo
model = modelo.fit(predictors_train, target_train)

In [ ]:
model.__dict__

# **6.Avaliação**

In [ ]:
#matriz de confusão
target_predicted = model.predict(predictors_test)

confusion_matrix = confusion_matrix(target_test, target_predicted)

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="coolwarm", cbar=True)
plt.xlabel("Previsões")
plt.ylabel("Valores Reais")
plt.title("Matriz de Confusão")
plt.show()

In [ ]:
#acurácia
accuracy = accuracy_score(target_test, target_predicted)
print(round(accuracy*100,2),"%")

# **conclusão**

o modelo de machine learnig demonstrou-se eficiente com uma acurácia de 99,97%, porém ainda acho que a espaço para melhoria se obtivesemos dados com a variavel temporal mais precisa ou até mesmo a data de iniciação desta poderia ser feito um modelo mais preciso.


